## Mode Decomposition Application

In [7]:
using LinearAlgebra
# using Plots
using PyCall
using PyPlot
using DataFrames
using CSV
using Symbolics
using Integrals
# plotlyjs()
matplotlib.use("TkAgg")

### Functions for all Integral Methods:

#### Trapezoidal and Simpson Methods
These integration methods implement the following formulas:
1. Trapezoidal: $\frac{h}{2} ( (f_1 +2f_2  \dots 2f_{n-1} +f_n)) $
2. Simpson 1/3: $\frac{h}{3} (f_1 +4f_2 +2f_3 +4f_4 +\dots +2f_{n-2}+4f_{n-1} +f_n) $
2. Simpson 3/8: $\frac{3h}{8} (f_1 +3f_2 +3f_3 +2f_4 +\dots+ 2f_{n-3} +3f_{n-2}+3f_{n-1} +f_n) $

In [210]:
function Integrate_Trapezoidal(f,h,lim)
    xᵢ= lim[1]:h:lim[2]
    intg=f(xᵢ[1])+f(xᵢ[end])

    for i in xᵢ[2:end-1]
        intg += 2f(i)
    end

    return intg*h/2
    
end

function simpson1_3(f,h,lim)
    xᵢ= lim[1]:h:lim[2]
    intg=f(xᵢ[1])+f(xᵢ[end])

    for i in xᵢ[2:2:end-1]
        intg += 4f(i) + 2f(i+h)
    end

    return intg*h/3
end

function simpson3_8(f,h,lim)
    xᵢ= lim[1]:h:lim[2]
    intg=f(xᵢ[1])+f(xᵢ[end])

    for i in xᵢ[2:3:end-2]
        intg += 3f(i) 
    end
    for i in xᵢ[3:3:end-1]
        intg += 3f(i) 
    end
    for i in xᵢ[4:3:end-3]
        intg += 2f(i) 
    end

    return intg*3h/8
end


simpson3_8 (generic function with 1 method)

#### Gauss Legendre Methods
These functions implement Gauss Legendre methods for 2,4,8 points. The optional arguments `t_func` and `dx_dt`are to transform x and $\frac{dx}{dt}$ respectively such that the limits of the integral are [-1,1] so that these methods work as expected. 

In [212]:

function Gauss_Legendre_2p(f::Function;t_func=identity,dx_dt=1, h= nothing, lim = nothing)
    Aᵢ = [ 1,1]
    xᵢ = [ -0.5773502692,0.5773502692]
   dot(Aᵢ,f.(t_func.(xᵢ)))*dx_dt 
end
function Gauss_Legendre_4p(f::Function;t_func=identity,dx_dt=1, h= nothing, lim = nothing)
    Aᵢ = [ 0.3478548451 , 0.6521451549, 0.6521451549, 0.3478548451 ]
    xᵢ = [ -0.8611363116 , -0.3394810436, 0.3394810436, 0.8611363116]
   dot(Aᵢ,f.(t_func.(xᵢ)))*dx_dt
end
function Gauss_Legendre_8p(f::Function;t_func=identity,dx_dt=1, h= nothing, lim = nothing)
    Aᵢ = [ 0.1012285363,0.2223810345,0.3137066459,0.3626837834,
           0.1012285363,0.2223810345,0.3137066459,0.3626837834]
    xᵢ = [ -0.9602898565,-0.7966664774,-0.5255324099,-0.1834346425,
            0.9602898565, 0.7966664774, 0.5255324099, 0.1834346425]
   dot(Aᵢ,f.(t_func.(xᵢ)))*dx_dt
end

Gauss_Legendre_8p (generic function with 1 method)

#### Spline Integration Method
Equal spacing is assumed for spline integration. First, we obtain the spline coefficients $a_i$, $b_i$, $c_i$ and $d_i$ as:<br>
1.  $a_i = \frac{S_{i+1}-S_i}{6h_i}$
2.  $b_i = \frac{S_i}{2}$
3.  $c_i = \frac{y_{i+1}-y_i}{h_i} - \frac{2h_iS_{i}+h_iS_{i+1}}{6}$
4. $d_i = y_i$

The solution for S is obtained by solving the matrix equation `AS = Y` where A is a tridiagonal matrix with elements $(h,2h,h)$ 

<br> Using these coefficients, the integral is approximated as: $\frac{h^4}{4}\sum_0^{n-1}a_i + \frac{h^3}{3}\sum_0^{n-1}b_i + \frac{h^2}{2}\sum_0^{n-1}c_i+ {h}\sum_0^{n-1}d_i$

In [203]:
function spline_coeff(f::Function,n,lim;verbose=false)
    Y = f.(range(lim[1],lim[2],length=n+1))
    h= Y[2]-Y[1]

    if verbose
        print(h,' ',n ,'\n')
        
    end

    # S=0 AND Sₙ=0
    A = Tridiagonal(repeat([h],n-2),repeat([2h],n-1),repeat([h],n-2))

    
    if verbose
        print(size(A),'\n')
        print(size(Y),'\n')
        
    end
    S = A\(Y[2:end-1])
    
    S = [0;S;0]
    # Y = [Y;f₁(lim[2])]
    if verbose
        print(size(S),'\n')
        print(size(Y),'\n')
        
    end
    aᵢ=zeros(n)
    bᵢ=zeros(n)
    cᵢ=zeros(n)
    dᵢ=zeros(n)
    for i in 1:(n)
        aᵢ[i] = (S[i+1]-S[i])/(6h)
        bᵢ[i] = S[i]/2
        cᵢ[i] = (Y[i+1]-Y[i])/h - (2h*S[i]+h*S[i+1])/6
        dᵢ[i] = Y[i]
    end
    return aᵢ,bᵢ,cᵢ,dᵢ,h
end

function spline_integration(f::Function,n,lim;verbose=false)
    
    aᵢ,bᵢ,cᵢ,dᵢ,h = spline_coeff(f,n,lim,verbose=verbose)
    
    intg = h^4/4*sum(aᵢ) + h^3/3*sum(bᵢ) + h^2/2*sum(cᵢ) + h*sum(dᵢ) 
    
    return intg,h
    
end

spline_integration (generic function with 1 method)

### Testing and Comparison

#### Definitions for Functions and their Analytical Integrals

In [141]:
function f₁(x)
    exp(x)cos(x)
    
end

function ∫f₁(x)
    0.5*exp(x)*(sin(x)+cos(x))
end 

function f₃(x)
    if x<0
        return exp(2x)
    else
        x-2cos(x)+4
    end
end

function ∫f₃(x)
    if x<0
        return exp(2x)/2
    else
        x^2/2-2sin(x)+4x
    end
end

∫f₃ (generic function with 1 method)

##### Function for Plotting log(E) vs log(h) for several Integration Methods

In [240]:
function Integral_Test(f,∫f,N,lim,t_func,dx_dt)
    True_Int = ∫f(lim[2]) - ∫f(lim[1])
    plt.close("all")
    
    for Int_Func in [Integrate_Trapezoidal,simpson1_3,simpson3_8]
        h_arr=zeros(length(N))
        Err_arr=zeros(length(N))

        for (n,i) in zip(N,1:length(N))
            h₁ = (lim[2]-lim[1])/n
            intg = Int_Func(f,h₁,lim)
            Err = abs(True_Int - intg)
        
            Err_arr[i] = Err
            h_arr[i] = h₁
        end
        plt.plot(h_arr,Err_arr)
    end

    for GL_Func in [Gauss_Legendre_2p,Gauss_Legendre_4p,Gauss_Legendre_8p]
        h_arr=zeros(length(N))
        Err_arr=zeros(length(N))

        for (n,i) in zip(N,1:length(N))
            h₁ = (lim[2]-lim[1])/n
            intg = GL_Func(f,t_func=t_func,dx_dt=dx_dt)
            Err = abs(True_Int - intg)
        
            Err_arr[i] = Err
            h_arr[i] = h₁
        end
        plt.plot(h_arr,Err_arr)
    end

    for Spline_Func in [spline_integration]
        h_arr=zeros(length(N))
        Err_arr=zeros(length(N))

        for (n,i) in zip(N,1:length(N))
            intg,h₁ = Spline_Func(f,n,lim)
            Err = abs(True_Int - intg)
            h₁ = (lim[2]-lim[1])/n
        
            Err_arr[i] = Err
            h_arr[i] = h₁
        end
        plt.plot(h_arr,Err_arr)
    end

    legend(string.([Integrate_Trapezoidal,simpson1_3,simpson3_8,Gauss_Legendre_2p,Gauss_Legendre_4p,Gauss_Legendre_8p,spline_integration]))
    xlabel("h")
    ylabel("Deviation from True Value")
    title("Error for Trapezoidal Method for "*string(f))
    grid()
    plt.show()
end


Integral_Test (generic function with 3 methods)

#### 1. $f₁(x) = e^xcos(x)$

For Gauss Legendre methods, transform x as: $t=\frac{4x}{\pi}-1$ such that the new limits are \[-1,1\] <br>
Therefore $x=(t+1)\frac{\pi}{4}$ and $dx = \frac{\pi}{4}dt$ <br>
$\frac{\pi}{4}\int_{-1}^1 e^{(t+1)\frac{\pi}{4}}cos((t+1)\frac{\pi}{4})dt$

In [247]:
N = 5:1:501
lim₁ = [0,π/2]
function Transform_for_GL(x)
    (x+1)*π/4
end
Integral_Test(f₁,∫f₁,N,lim₁,Transform_for_GL,π/4)

#### Notes

Looking at the graphs, we see simpson 3/8 converging slower than simpson 1/3 and trapezoidal methods which converge at a similar rate. Spline integration converges faster than simpson 3/8 but slower than other two. The simpson integrals display oscillatory behaviour, I'm not sure why <br>
The Gauss Legendre methods get closer to the true value of the integral for larger number of points used and are the most accurate for this function

#### 2. $f_2(x) = e^x$

For Gauss Legendre methods, transform x as: $t=\frac{x-1}{2}$ such that the new limits are \[-1,1\] <br>
Therefore $x=(2t+1)$ and $dx = 2dt$ <br>
$2\int_{-1}^1 e^{(2t+1)}dt$

In [246]:
N = 5:1:501
lim₁ = [-1,3]
function Transform_for_GL(x)
    2x+1
end
Integral_Test(f₁,∫f₁,N,lim₁,Transform_for_GL,2)

#### 3. $f_3(x) = e^{2x} $ for x\<0 and $f_3(x) = x-cos(x)+4 $ for x $\geq$ 0

In [244]:
N = 5:1:501
lim₁ = [-1,1]

Integral_Test(f₁,∫f₁,N,lim₁,identity,1)

#### Notes
For the last two functions we see something different, the spline integration gets worse for larger number of points, which is expected since interpolation with very large number of points can result in numerical instabilities

#### Better Integral Methods
We can improve our integral approximation by using methods like Romberg integration or using higher order Newton Cotes formulaes

### Applications
This will be done after we have discussed PDEs